In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from local.torch_basics import *
from local.test import *
from local.basics import *
from local.data.all import *
from local.vision.core import *
from local.notebook.showdoc import show_doc
from local.audio.core import *
from local.audio.augment import *
from local.vision.learner import *
from local.vision.models.xresnet import *
from local.metrics import *
from local.callback.schedule import *
import torchaudio
from fastprogress import progress_bar as pb
from numba import njit, prange
from rocket import ROCKET, ROCKETMAX

In [ ]:
PATH_25 = Config()['data_path'] / '25_speakers'
x = AudioGetter("", recurse=True, folders=None)
files_25  = x(PATH_25)
oa = OpenAudio(files_25)

In [ ]:
files_25

(#3417) [/home/jupyter/.fastai/data/25_speakers/id08870/u-6MoWmZzeU/00081.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00061.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00062.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00060.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00050.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00036.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00038.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00046.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00035.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00053.wav...]

In [ ]:
def AudioBlock(cls=AudioItem): return TransformBlock(type_tfms=cls.create, batch_tfms=IntToFloatTensor)

In [ ]:
def extract_data(dbunch):
    train_dl, valid_dl = dbunch
    sigs_train, labels_train = [],[]
    for batch in pb(train_dl):
        sigs_train.append(batch[0])
        labels_train.append(batch[1])
    sigs_valid, labels_valid = [],[]
    for batch in pb(valid_dl):
        sigs_valid.append(batch[0])
        labels_valid.append(batch[1])
    sigs_train = torch.cat(sigs_train, axis=0).cpu().numpy()
    labels_train = torch.cat(labels_train, axis=0).cpu().numpy()
    sigs_valid = torch.cat(sigs_valid, axis=0).cpu().numpy()
    labels_valid = torch.cat(labels_valid, axis=0).cpu().numpy()
    return sigs_train, labels_train, sigs_valid, labels_valid

In [ ]:
auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_items=get_audio_files, 
                 splitter=RandomSplitter(),
                 get_y=lambda x: str(x).split('/')[-3][3:])

In [ ]:
def Normalize(m, s):
    def _inner(ai:AudioItem)->AudioItem:
        sig = ai.sig.clone()
        norm_sig = (sig - m)/(s + 1e-8)
        return AudioItem((norm_sig, ai.sr, ai.path))
    return _inner

In [ ]:
norm = Normalize(1e-05, 0.0566)

In [ ]:
total_mean = 0
total_std = 0
total_mean2 = 0
total_std2 = 0
num_files=len(files_25[0:1000])
for i in pb(range(num_files)):
    total_mean += oa(i).sig.mean()
    total_std  += oa(i).sig.std()
    total_mean2 += norm(oa(i)).sig.mean()
    total_std2  += norm(oa(i)).sig.std()
mn, std = total_mean/num_files, total_std/num_files
norm_mn, norm_std = total_mean2/num_files, total_std2/num_files

In [ ]:
mn

tensor(-3.1599e-06)

In [ ]:
std

tensor(0.0492)

In [ ]:
norm_mn, norm_std

(tensor(-3.2489e-06), tensor(0.9993))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rocket = ROCKET(1, 32000, n_kernels=1000, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
def GetSig():
    def _inner(ai:AudioItem)->AudioItem:
        return ai.sig
    return _inner

In [ ]:
crop_2000ms = CropSignal(2000)
tfms = [crop_2000ms, norm, GetSig(),]
gpu_tfms = [Cuda(), rocket]
dbunch = auds.databunch(PATH_25, item_tfms=tfms,  batch_tfms=gpu_tfms, bs=512)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 9.73 s, sys: 7.28 s, total: 17 s
Wall time: 39 s


In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
x_train.shape

(2560, 2000)

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 8.44 s, sys: 428 ms, total: 8.86 s
Wall time: 2.24 s


0.6471449487554904

In [ ]:
rocketmax = ROCKETMAX(1, 32000, n_kernels=1000, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
gpu_tfms_max = [Cuda(), rocketmax]
dbunch = auds.databunch(PATH_25, item_tfms=tfms,  batch_tfms=gpu_tfms_max, bs=512)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 47.6 s, sys: 27.9 s, total: 1min 15s
Wall time: 1min 45s


In [ ]:
x_train.shape

(2560, 1, 12000)

In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 9.03 s, sys: 568 ms, total: 9.6 s
Wall time: 2.9 s


0.7759882869692533

In [ ]:
x_train.shape


(2688, 12000)

In [ ]:
x_train[0][0:14]

array([-14.84348   ,  17.285114  ,  -0.8216262 ,   2.5483599 ,
         0.83445895,   0.7804303 ,   0.6524266 ,   0.3226009 ,
         0.21203078,   0.15501806,   0.11591017,   0.09156588,
         0.07366107, -22.79066   ], dtype=float32)

# Applying the same code to 11 speaker problem

In [ ]:
PATH_11 = Config()['data_path'] / '11_speakers'
x = AudioGetter("", recurse=True, folders=None)
files_11  = x(PATH_11)
oa = OpenAudio(files_11)

In [ ]:
files_11

(#1916) [/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00344.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00199.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00233.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00323.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00266.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00241.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00203.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00276.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00242.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00245.wav...]

In [ ]:
norm = Normalize(-3e-06, 0.0492)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rocket = ROCKET(1, 32000, n_kernels=1000, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
def GetSig():
    def _inner(ai:AudioItem)->AudioItem:
        return ai.sig
    return _inner

In [ ]:
crop_2000ms = CropSignal(2000)
tfms = [crop_2000ms, norm, GetSig(),]
gpu_tfms = [Cuda(), rocket]
dbunch = auds.databunch(PATH_11, item_tfms=tfms,  batch_tfms=gpu_tfms, bs=128)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 8.61 s, sys: 3.57 s, total: 12.2 s
Wall time: 19 s


In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
x_train.shape

(1408, 2000)

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 1.86 s, sys: 88 ms, total: 1.94 s
Wall time: 529 ms


0.8537859007832899

In [ ]:
rocketmax = ROCKETMAX(1, 32000, n_kernels=1000, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
gpu_tfms_max = [Cuda(), rocketmax]
dbunch = auds.databunch(PATH_11, item_tfms=tfms,  batch_tfms=gpu_tfms_max, bs=128)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 16.7 s, sys: 7.58 s, total: 24.2 s
Wall time: 28.5 s


In [ ]:
x_train.shape

(1408, 1, 6000)

In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 2.02 s, sys: 76 ms, total: 2.1 s
Wall time: 566 ms


0.8720626631853786